## Pull in UDFs

In [ ]:
%run nb_udfs


## Run Dashboards

In [ ]:
workspace = 'Fabric of Middle-Earth' #have to escape the & symbol using %26% 
lakehouse = 'lh_the_hoard'
dashboard_table = 'dimDashboards'


In [ ]:
#get a list of all dashboards and load to a table in the lakehouse
response = _base_api(
        request=f"/v1.0/myorg/admin/dashboards",
        method="get"
    )
df_dashboards = pd.json_normalize(response.json()['value'])
df_dashboards = spark.createDataFrame(df_dashboards)
df_dashboards = df_dashboards.drop("users","subscriptions")
#df_dashboards.printSchema()
#print(df_dashboards.columns)


In [ ]:
#creates a slowly changing dimension so we can keep an eye on any deleted dashboards
dashboardPath = udf_GetFilePath(workspace, lakehouse, dashboard_table)
#print(dashboardPath)
naturalKeyColumnList = ['id']
primaryKeyColumnName = "tableId"
returnVal = udf_UpsertDimension(df_dashboards,2,dashboardPath,naturalKeyColumnList,primaryKeyColumnName,True)
print(returnVal)